# Agent with tools
This notebook illustrates how to build a simple agent capable of using tools. 
We'll use crewAI to illustrate how agentics implements logical transduction

## Initialize environment

Note: Before executing this notebook on colab, copy your .env file in /content/drive/MyDrive/.env or otherwise you'll be prompted to enter your GEMINI_API_KEY all the times.

In [1]:
! uv pip install agentics-py

import os
import sys
from getpass import getpass

from dotenv import find_dotenv, load_dotenv

CURRENT_PATH = ""

IN_COLAB = "google.colab" in sys.modules
print("In Colab:", IN_COLAB)


if IN_COLAB:
    CURRENT_PATH = "/content/drive/MyDrive/"
    # Mount your google drive
    from google.colab import drive
    drive.mount("/content/drive")
    from google.colab import userdata

    os.environ["GEMINI_API_KEY"] = userdata.get("GEMINI_API_KEY")
else:
    load_dotenv(find_dotenv())

if not os.getenv("GEMINI_API_KEY"):
    os.environ["GEMINI_API_KEY"] = getpass("Enter your GEMINI_API_KEY:")

Using Python 3.12.9 environment at: /Users/gliozzo/Code/agentics911/agentics/.venv
Audited 1 package in 16ms
In Colab: False


## Create a new tool
Wrap the Duck Duck Go Search API call into a CrewAI tool using the @tool decorator. Similarly you can turn any function of your choice into a tool. 

In [2]:
from agentics import AG
from crewai.tools import tool
from ddgs import DDGS


## Define a Crew AI tool to get news for a given date using the DDGS search engine
@tool("fetch_data_async")
def web_search(query: str) -> str:
    """Fetch web search results for the given query using DDGS."""
    return str(DDGS().text(query, max_results=10))

## Define an Agent

Use crew AI agent directly create a document searcher agent, who is instructed to use the web_search tool defined above. Note that reasoning steps and other parameters can be customized. 


In [ ]:
from crewai import Agent

doc_agent = Agent(
    role="Doc Searcher",
    goal="Find answers to questions from the user using the available web search tool.",
    backstory="A helpful assistant for extensive web search reports.",
    tools=[web_search],
    reasoning=True,
    reasoning_steps=5,
    verbose=True,
    llm=AG.get_llm_provider("gemini")
)

## Define a Task

In [4]:
from crewai import Task

doc_task = Task(
    description="""Your task is to perform an extensive web search about
    the following question {question} and return a document providing answers to 
    the questions that explore several interesting aspects, each of them supported 
    by pertinent information from web search.  """,
    expected_output="""A markdown document with introduction and different sections""",
    agent=doc_agent,
    markdown=True,
)

## Define a Crew

Crews are enviroments in which tasks are assigned to different agents following different policies. In this simple example we define a crew with a single agent and single task. 

In [5]:
from crewai import Crew

crew = Crew(
    agents=[doc_agent],
    tasks=[doc_task],
    verbose=True,
)

## Kickoff the crew

Crews are ultimatelly analoguous to very sophisticated function definitions, whose inputs are json dictonaries. In order to put them in action you should kickoff them providing input parameters.

In the Agent Trace stack you can see the plan made by the agent and the different execution steps which might involve tool calls.

In [6]:
result = crew.kickoff(inputs={"question": input("Ask your question> ")})

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: f7c84e12-b4ab-4540-8bd0-7dbdb0bae1bb                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭─────────────────────────────────────────────── 🧠 Reasoning Plan ───────────────────────────────────────────────╮
│                                                                                                                 │
│  Okay, here's my plan to tackle the task of researching Alfio Gliozzo and creating a comprehensive document     │
│  about him:                                                                                                     │
│                                                                                                                 │
│  **1. Understanding the Task:**                                                                                 │
│                                                                                                                 │
│  As Doc Searcher, I understand that my primary goal is to conduct an in-depth investigation into Alfio Gliozzo  │
│  using web search and compile the findings into a well-structured document. The document should not only        │
│  answer the basic "who is Alfio Gliozzo?" question but also explore interesting facets of his life and work,    │
│  supported by evidence from web searches. The key is to go beyond a simple biography and uncover unique or      │
│  noteworthy aspects.                                                                                            │
│                                                                                                                 │
│  **2. Key Steps:**                                                                                              │
│                                                                                                                 │
│  *   **Initial Search and Information Gathering:** I will start with a broad web search using the query "Alfio  │
│  Gliozzo." This initial search will aim to identify potential sources like personal websites, professional      │
│  profiles (LinkedIn, Google Scholar, etc.), news articles, publications, and interviews.                        │
│  *   **Identifying Interesting Aspects:** Based on the initial search results, I'll identify potential areas    │
│  of focus for the document. This could include his profession, significant achievements, research interests,    │
│  affiliations, or any controversies or notable events associated with him. I'll prioritize aspects that have    │
│  sufficient information available online to support a detailed discussion.                                      │
│  *   **Targeted Searches:** For each identified aspect, I will conduct more specific web searches to gather     │
│  detailed information. For example, if he's a researcher, I'll search for his publications, research grants,    │
│  and citations. If he's involved in a particular organization, I'll search for his role and contributions.      │
│  *   **Information Synthesis and Document Creation:** I will synthesize the information gathered from the       │
│  various web searches into a coherent and well-organized document. The document will have an introduction,      │
│  separate sections for each interesting aspect, and proper citations or references to the sources of            │
│  information.                                                                                                   │
│  *   **Review and Refinement:** After creating the initial draft, I will review the document for accuracy,      │
│  clarity, and completeness. I will also perform additional searches to fill any gaps in information or address  │
│  any inconsistencies.                                                                                           │
│                                                                                                                 │
│  **3. Addressing Potential Challenges:**                                                                        │
│                                                        

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Doc Searcher                                                                                            │
│                                                                                                                 │
│  Task: Your task is to perform an extensive web search about                                                    │
│      the following question Who is alfio gliozzo and return a document providing answers to                     │
│      the questions that explore several interesting aspects, each of them supported                             │
│      by pertinent information from web search.                                                                  │
│                                                                                                                 │
│  Reasoning Plan:                                                                                                │
│  Okay, here's my plan to tackle the task of researching Alfio Gliozzo and creating a comprehensive document     │
│  about him:                                                                                                     │
│                                                                                                                 │
│  **1. Understanding the Task:**                                                                                 │
│                                                                                                                 │
│  As Doc Searcher, I understand that my primary goal is to conduct an in-depth investigation into Alfio Gliozzo  │
│  using web search and compile the findings into a well-structured document. The document should not only        │
│  answer the basic "who is Alfio Gliozzo?" question but also explore interesting facets of his life and work,    │
│  supported by evidence from web searches. The key is to go beyond a simple biography and uncover unique or      │
│  noteworthy aspects.                                                                                            │
│                                                                                                                 │
│  **2. Key Steps:**                                                                                              │
│                                                                                                                 │
│  *   **Initial Search and Information Gathering:** I will start with a broad web search using the query "Alfio  │
│  Gliozzo." This initial search will aim to identify potential sources like personal websites, professional      │
│  profiles (LinkedIn, Google Scholar, etc.), news articles, publications, and interviews.                        │
│  *   **Identifying Interesting Aspects:** Based on the initial search results, I'll identify potential areas    │
│  of focus for the document. This could include his profession, significant achievements, research interests,    │
│  affiliations, or any controversies or notable events associated with him. I'll prioritize aspects that have    │
│  sufficient information available online to support a detailed discussion.                                      │
│  *   **Targeted Searches:** For each identified aspect, I will conduct more specific web searches to gather     │
│  detailed information. For example, if he's a researcher, I'll search for his publications, research grants,    │
│  and citations. If he's involved in a particular organization, I'll search for his role and contributions.      │
│  *   **Information Synthesis and Document Creation:** I will synthesize the information gathered from the       │
│  various web searches into a coherent and well-organize

Output()

...

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  [{'title': 'Alfio Gliozzo | IBM | 123 Publications | 1512 Citations', 'href':                                  │
│  'https://scispace.com/authors/alfio-gliozzo-111s34lqdy', 'body': 'Alfio Gliozzo is an academic researcher      │
│  from IBM. The author has contributed to research in topics: Computer science & Question answering.'},          │
│  {'title': 'Alfio Gliozzo | Scholar Profiles and Rankings', 'href':                                             │
│  'https://scholargps.com/scholars/81406211210414/alfio-gliozzo', 'body': 'Alfio Gliozzo ; Publication Count.    │
│  88 ; Predicted Citations. 1,060 ; Predicted h-index. 18.'}, {'title': 'Alfio Gliozzo', 'href':                 │
│  'https://openreview.net/profile?id=~Alfio_Gliozzo1', 'body': 'Publications . Translucent answer predictions    │
│  in multi-hop reading ... Learning Relational Representations by Analogy using Hierarchical Siamese Networks ·  │
│  Gaetano ...'}, {'title': 'Alfio Massimiliano Gliozzo - Chief Science Catalyst', 'href':                        │
│  'https://www.linkedin.com/in/gliozzo', 'body': 'With a PhD in Artificial Intelligence earned in 2005, Alfio    │
│  has an impressive portfolio, having authored over 150 scientific publications , including books , ...'},       │
│  {'title': 'List of books by author Alfio Gliozzo', 'href':                                                     │
│  'https://www.thriftbooks.com/a/alfio-gliozzo/3208093/?srsltid=AfmBOoo8nkF-c-k_EtNyKYqwXGb9R4OK0pmBYiP2TY_bxVT  │
│  KZeU6eyo9', 'body': 'Looking for a book by Alfio Gliozzo ? Alfio Gliozzo wrote Semantic Domains in             │
│  Computational Linguistics, which can be purchased at a lower price at ...'}, {'title': 'Alfio Gliozzo',        │
│  'href': 'https://www.knowledgegraph.tech/blog/speakers/alfio-gliozzo/', 'body': 'He is author of more than 60  │
│  scientific publications in AI conferences, journals and books , and he holds dozens of patents for IBM.        │
│  Extending Knowledge ...'}, {'title': 'Alfio Massimiliano Gliozzo - Home - ACM Digital Library', 'href':        │
│  'https://dl.acm.org/profile/81100464565', 'body': 'Natural language processing Knowledge representation and    │
│  reasoning Machine learning Artificial intelligence Computing methodologies Document filtering ...'},           │
│  {'title': 'Improving tex...                                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Doc Searcher                                                                                            │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  ```markdown                                                                                                    │
│  # Alfio Gliozzo: A Profile                                                                                     │
│                                                                                                                 │
│  ## Introduction                                                                                                │
│                                                                                                                 │
│  Alfio Gliozzo is a Principal Researcher at IBM Research AI, where he also serves as a Technical Program        │
│  Manager and Chief Science Catalyst for Data and AI Platforms. He is a scientist, technologist, and manager     │
│  with extensive experience in Artificial Intelligence, specifically Machine Learning and Natural Language       │
│  Processing.                                                                                                    │
│                                                                                                                 │
│  ## Role at IBM Research AI                                                                                     │
│                                                                                                                 │
│  At IBM Research AI, Alfio Gliozzo plays a key role in enhancing IBM products with AI solutions, especially in  │
│  the realm of knowledge representation and reasoning. He is the leader of the Knowledge Area of IBM Research    │
│  AI tech, and his work focuses on integrating cutting-edge AI solutions into IBM's offerings. He has been with  │
│  IBM for a significant period, accumulating over 17 years of experience in the field.                           │
│                                                                                                                 │
│  ## Research Interests and Publications                                                                         │
│                                                                                                                 │
│  Alfio Gliozzo's research interests lie primarily in Artificial Intelligence, with a strong focus on Natural    │
│  Language Processing, Knowledge Representation and Reasoning, and Machine Learning. He has an impressive        │
│  portfolio of over 150 scientific publications, including books, papers, and patents. His publications span     │
│  various AI conferences and journals. According to scispace.com, he has 123 publications with 1512 citations.   │
│  He was also a part of the DeepQA team that developed Watson.                                                   │
│                                                                                                                 │
│  ## Education and Background                                                                                    │
│                                                                                                                 │
│  Alfio Gliozzo holds a Ph.D. in Artificial Intelligence, earned in 2005.                                        │
│                                                                                                                 │
│  ## Other Notable Aspects                              

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 61e32aea-f18f-48cc-8f03-b1b62e3505b9                                                                     │
│  Agent: Doc Searcher                                                                                            │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: f7c84e12-b4ab-4540-8bd0-7dbdb0bae1bb                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: ```markdown                                                                                      │
│  # Alfio Gliozzo: A Profile                                                                                     │
│                                                                                                                 │
│  ## Introduction                                                                                                │
│                                                                                                                 │
│  Alfio Gliozzo is a Principal Researcher at IBM Research AI, where he also serves as a Technical Program        │
│  Manager and Chief Science Catalyst for Data and AI Platforms. He is a scientist, technologist, and manager     │
│  with extensive experience in Artificial Intelligence, specifically Machine Learning and Natural Language       │
│  Processing.                                                                                                    │
│                                                                                                                 │
│  ## Role at IBM Research AI                                                                                     │
│                                                                                                                 │
│  At IBM Research AI, Alfio Gliozzo plays a key role in enhancing IBM products with AI solutions, especially in  │
│  the realm of knowledge representation and reasoning. He is the leader of the Knowledge Area of IBM Research    │
│  AI tech, and his work focuses on integrating cutting-edge AI solutions into IBM's offerings. He has been with  │
│  IBM for a significant period, accumulating over 17 years of experience in the field.                           │
│                                                                                                                 │
│  ## Research Interests and Publications                                                                         │
│                                                                                                                 │
│  Alfio Gliozzo's research interests lie primarily in Artificial Intelligence, with a strong focus on Natural    │
│  Language Processing, Knowledge Representation and Reasoning, and Machine Learning. He has an impressive        │
│  portfolio of over 150 scientific publications, including books, papers, and patents. His publications span     │
│  various AI conferences and journals. According to scispace.com, he has 123 publications with 1512 citations.   │
│  He was also a part of the DeepQA team that developed Watson.                                                   │
│                                                                                                                 │
│  ## Education and Background                                                                                    │
│                                                                                                                 │
│  Alfio Gliozzo holds a Ph.D. in Artificial Intelligence, earned in 2005.                                        │
│                                                       

## Display the results as markdown

In [7]:
from IPython.display import Markdown, display

display(Markdown(result.raw))

```markdown
# Alfio Gliozzo: A Profile

## Introduction

Alfio Gliozzo is a Principal Researcher at IBM Research AI, where he also serves as a Technical Program Manager and Chief Science Catalyst for Data and AI Platforms. He is a scientist, technologist, and manager with extensive experience in Artificial Intelligence, specifically Machine Learning and Natural Language Processing.

## Role at IBM Research AI

At IBM Research AI, Alfio Gliozzo plays a key role in enhancing IBM products with AI solutions, especially in the realm of knowledge representation and reasoning. He is the leader of the Knowledge Area of IBM Research AI tech, and his work focuses on integrating cutting-edge AI solutions into IBM's offerings. He has been with IBM for a significant period, accumulating over 17 years of experience in the field.

## Research Interests and Publications

Alfio Gliozzo's research interests lie primarily in Artificial Intelligence, with a strong focus on Natural Language Processing, Knowledge Representation and Reasoning, and Machine Learning. He has an impressive portfolio of over 150 scientific publications, including books, papers, and patents. His publications span various AI conferences and journals. According to scispace.com, he has 123 publications with 1512 citations. He was also a part of the DeepQA team that developed Watson.

## Education and Background

Alfio Gliozzo holds a Ph.D. in Artificial Intelligence, earned in 2005.

## Other Notable Aspects

*   **Chief Science Catalyst:** His role as Chief Science Catalyst highlights his ability to drive innovation and translate research into practical applications.
*   **Technical Program Manager:** As a Technical Program Manager, he is involved in the planning and execution of research projects.
*   **Philosophical Interests:** His Twitter profile indicates an interest in philosophy, suggesting a broad intellectual curiosity that extends beyond technical domains.
*   **Recognition:** Alfio Gliozzo's work has been recognized through numerous publications and citations, solidifying his position as a leading researcher in the field of AI.
*   **Speaker:** Alfio Gliozzo has been a speaker at various conferences and events, sharing his expertise and insights on AI-related topics.

## Conclusion

Alfio Gliozzo is a distinguished researcher and leader in the field of Artificial Intelligence. His contributions to IBM Research AI, particularly in the areas of knowledge representation, natural language processing, and machine learning, have made a significant impact. With a strong publication record and a passion for innovation, he continues to be a driving force in the advancement of AI technology.
```